# Tokenizador simples (Cap 2)

**Descrição:**
Um tokenizador é o componente responsável por transformar texto bruto em unidades menores chamadas *tokens*. Esses tokens podem ser palavras, subpalavras, caracteres ou símbolos especiais (como pontuação). Como modelos de linguagem não conseguem processar texto diretamente, o tokenizador atua como a ponte entre o texto humano e as representações numéricas que o modelo consegue entender. Esse processo é o primeiro passo fundamental no preparo dos dados para o treinamento e uso de modelos de linguagem de grande escala (LLMs). 

**Objetivo:**
O principal objetivo de um tokenizador é padronizar e converter o texto em uma sequência estruturada de tokens que possa ser posteriormente mapeada para identificadores numéricos (*token IDs*) e, então, para vetores de embeddings. Dessa forma, o tokenizador viabiliza que o modelo aprenda padrões da linguagem, como estrutura, contexto e significado, sendo essencial para tarefas como geração de texto, classificação e compreensão de linguagem natural. 

**Funcionamento:**

![Exemplo de funcionamento de um tokenizador](../../images/01_tokenizador_funcionamento.png)

*Figura 1 — Exemplo do funcionamento de um tokenizador: texto → tokens → token IDs.*

O tokenizador transforma texto em tokens e tokens em números, permitindo que o modelo de linguagem processe a informação.


## Implementação de um tokenizador simples

In [1]:
import re
from dataclasses import dataclass
from typing import Dict, List, Tuple

@dataclass
class TokenizationResult:
    """Estrutura para guardar resultados intermediários da tokenização."""
    raw_text: str
    split_raw: List[str]
    tokens: List[str]
    vocab: Dict[str, int]
    token_ids: List[int]


def tokenizar_passo_a_passo(texto: str) -> TokenizationResult:
    """
    Tokeniza um texto de forma didática, mostrando o passo a passo.

    Estratégia (simples e didática):
    - Separar palavras e pontuação com regex
    - Remover itens vazios e espaços
    - Construir vocabulário (token -> id) ordenando alfabeticamente
    - Converter tokens em token IDs

    Parâmetros:
    ----------
    texto : str
        Texto a ser tokenizado.

    Retorno:
    -------
    TokenizationResult
        Estrutura contendo todas as saídas intermediárias e finais.

    Exceções:
    --------
    TypeError
        Se `texto` não for string.
    ValueError
        Se `texto` for uma string vazia após remoção de espaços.
    """
    if not isinstance(texto, str):
        raise TypeError("O parâmetro `texto` deve ser uma string (str).")

    if not texto.strip():
        raise ValueError("O parâmetro `texto` não pode ser vazio.")

    print("=== PASSO 0: Texto original ===")
    print(texto)
    print()

    # Regex didática:
    # - Captura pontuação básica em um grupo separado: [,.!?;:]
    # - Captura espaços (\s) como separadores
    padrao = r'([,.!?;:]|\s)'

    print("=== PASSO 1: Split bruto (inclui espaços e strings vazias) ===")
    split_raw = re.split(padrao, texto)
    print(split_raw)
    print()

    print("=== PASSO 2: Limpeza (remove espaços e itens vazios) ===")
    # Mantemos apenas partes com conteúdo (strip remove espaços)
    tokens = [p.strip() for p in split_raw if p.strip()]
    print(tokens)
    print()

    print("=== PASSO 3: Construção do vocabulário (token -> id) ===")
    # Ordenar garante reprodutibilidade (didático)
    tokens_unicos_ordenados = sorted(set(tokens))
    vocab = {tok: i for i, tok in enumerate(tokens_unicos_ordenados)}
    print("Tokens únicos (ordenados):", tokens_unicos_ordenados)
    print("Vocabulário:", vocab)
    print()

    print("=== PASSO 4: Conversão tokens -> token IDs ===")
    token_ids = [vocab[tok] for tok in tokens]
    print("Tokens:", tokens)
    print("Token IDs:", token_ids)
    print()

    return TokenizationResult(
        raw_text=texto,
        split_raw=split_raw,
        tokens=tokens,
        vocab=vocab,
        token_ids=token_ids,
    )


# =========================
# Exemplo de uso (pedido)
# =========================
texto_exemplo = "O tokenizador divide o texto."
resultado = tokenizar_passo_a_passo(texto_exemplo)

print(resultado.tokens)
print(resultado.token_ids)

=== PASSO 0: Texto original ===
O tokenizador divide o texto.

=== PASSO 1: Split bruto (inclui espaços e strings vazias) ===
['O', ' ', 'tokenizador', ' ', 'divide', ' ', 'o', ' ', 'texto', '.', '']

=== PASSO 2: Limpeza (remove espaços e itens vazios) ===
['O', 'tokenizador', 'divide', 'o', 'texto', '.']

=== PASSO 3: Construção do vocabulário (token -> id) ===
Tokens únicos (ordenados): ['.', 'O', 'divide', 'o', 'texto', 'tokenizador']
Vocabulário: {'.': 0, 'O': 1, 'divide': 2, 'o': 3, 'texto': 4, 'tokenizador': 5}

=== PASSO 4: Conversão tokens -> token IDs ===
Tokens: ['O', 'tokenizador', 'divide', 'o', 'texto', '.']
Token IDs: [1, 5, 2, 3, 4, 0]

['O', 'tokenizador', 'divide', 'o', 'texto', '.']
[1, 5, 2, 3, 4, 0]


### Análise do resultado:


#### PASSO 0 — Texto original

```text
O tokenizador divide o texto.
```

Neste ponto temos o **texto bruto**, exatamente como escrito por um humano.
Modelos de linguagem **não conseguem operar diretamente sobre texto**, então esse formato ainda não é utilizável pelo modelo.

---

#### PASSO 1 — Split bruto (inclui espaços e strings vazias)

```python
['O', ' ', 'tokenizador', ' ', 'divide', ' ', 'o', ' ', 'texto', '.', '']
```

**O que aconteceu aqui:**

* O texto foi dividido usando uma expressão regular que separa:

  * palavras
  * espaços (`' '`)
  * pontuação (`'.'`)
* Os separadores capturados pela regex **também aparecem na lista**, o que é intencional neste passo didático.

**Por que isso é importante?**

* Esse resultado mostra claramente que a tokenização inicial é “suja”.
* Espaços e strings vazias aparecem naturalmente e **precisam ser tratados**.
* Ajuda a entender que tokenização não é apenas um `split()` simples.

---

#### PASSO 2 — Limpeza (remove espaços e itens vazios)

```python
['O', 'tokenizador', 'divide', 'o', 'texto', '.']
```

**O que foi feito:**

* Removemos:

  * espaços
  * strings vazias
* Mantivemos:

  * palavras
  * pontuação como token independente

**Pontos importantes:**

* A pontuação (`.`) é mantida como token próprio
  → isso é fundamental em modelos de linguagem
* A capitalização foi preservada:

  * `"O"` ≠ `"o"`

Esse é o **conjunto final de tokens** que representa o texto.

---

#### PASSO 3 — Construção do vocabulário (token → id)

```python
Tokens únicos (ordenados): ['.', 'O', 'divide', 'o', 'texto', 'tokenizador']
Vocabulário: {
    '.': 0,
    'O': 1,
    'divide': 2,
    'o': 3,
    'texto': 4,
    'tokenizador': 5
}
```

**O que é o vocabulário:**

* Um mapeamento entre cada token único e um número inteiro
* Cada token recebe um **ID fixo**

**Por que ordenar alfabeticamente?**

* Garante reprodutibilidade
* Facilita entendimento didático
* Em implementações reais, a ordem costuma ser baseada em frequência

**Observações didáticas importantes:**

* `"O"` e `"o"` são tokens diferentes
* A pontuação faz parte do vocabulário
* O modelo não “entende” palavras — ele só vê números

---

#### PASSO 4 — Conversão de tokens para token IDs

```python
Tokens:    ['O', 'tokenizador', 'divide', 'o', 'texto', '.']
Token IDs: [1, 5, 2, 3, 4, 0]
```

**O que aconteceu:**

* Cada token foi substituído pelo seu ID correspondente no vocabulário
* A **ordem original do texto foi preservada**

**Resultado final:**

* O texto agora está em um formato **totalmente numérico**
* Esse é o formato que pode ser:

  * convertido em embeddings
  * usado como entrada em um modelo de linguagem

---

#### Conclusão

Esse exemplo mostra claramente que:

1. **Tokenizar não é apenas separar palavras**
2. O processo envolve:

   * separação
   * limpeza
   * construção de vocabulário
   * mapeamento para números
3. O modelo de linguagem:

   * **não vê texto**
   * **não vê palavras**
   * vê apenas **sequências de números**

## Uma nova execução (com outro texto)

In [2]:
texto_exemplo = "O tokenizador divide em pedaços menores."
resultado = tokenizar_passo_a_passo(texto_exemplo)

print(resultado.tokens)
print(resultado.token_ids)

=== PASSO 0: Texto original ===
O tokenizador divide em pedaços menores.

=== PASSO 1: Split bruto (inclui espaços e strings vazias) ===
['O', ' ', 'tokenizador', ' ', 'divide', ' ', 'em', ' ', 'pedaços', ' ', 'menores', '.', '']

=== PASSO 2: Limpeza (remove espaços e itens vazios) ===
['O', 'tokenizador', 'divide', 'em', 'pedaços', 'menores', '.']

=== PASSO 3: Construção do vocabulário (token -> id) ===
Tokens únicos (ordenados): ['.', 'O', 'divide', 'em', 'menores', 'pedaços', 'tokenizador']
Vocabulário: {'.': 0, 'O': 1, 'divide': 2, 'em': 3, 'menores': 4, 'pedaços': 5, 'tokenizador': 6}

=== PASSO 4: Conversão tokens -> token IDs ===
Tokens: ['O', 'tokenizador', 'divide', 'em', 'pedaços', 'menores', '.']
Token IDs: [1, 6, 2, 3, 5, 4, 0]

['O', 'tokenizador', 'divide', 'em', 'pedaços', 'menores', '.']
[1, 6, 2, 3, 5, 4, 0]


**Observação:** 

Como podemos verificar na execução acima: O tokenizador é determinístico para um dado texto, mas seus token IDs só fazem sentido dentro do vocabulário construído.